## Workbook on solving ODEs with sympy

Recall our ODE from the [skydiver problem](https://github.com/fherwig/physmath248_pilot/tree/master/examples/Skydiver.ipynb). 

$$ v' = kv^2 -g $$

In [ ]:
## Analytic Solution in implicit form, using SymPy.
import sympy as sp

f=sp.Function('v')
x=sp.Symbol('x')

g,k=sp.symbols('g k')
ODE = sp.Eq( sp.Derivative(f(x), x), k*f(x)**2 - g )

print("We wish to get sympy to solve the differential equation: ")
sp.pprint(ODE)


In [ ]:
print("It is perfectly acceptable to ask sympy to solve an ODE given in the traditional form above.")
sp.pprint(sp.dsolve(ODE))

In [ ]:
print("Or to rewrite it more traditionally: ")
ODE = sp.Eq( sp.Derivative(f(x), x) - k*f(x)**2 + g, 0 )
sp.pprint(ODE)
sp.pprint(sp.dsolve(ODE))

In [ ]:
print("Or even to neglect the equality on the right-hand side...")
ODE= sp.Derivative(f(x), x) - k*f(x)**2 + g
sp.pprint(ODE)
print("In this case, sympy **assumes** the =0 on the right hand side.")
sp.pprint(sp.dsolve(ODE))

In [ ]:
## Let's plot some solution curves. 

SOL = sp.dsolve(ODE)

SOLt = SOL.subs([(g, 9), (k,1)])
print("With g=9 and k=1 our solution curves are: \n")
sp.pprint(SOLt)

## We have too many variables for v to depend directly on x.
## Let's replace g and k with something reasonable. 


In [ ]:
## this expression still depends on a constant.  We need to extract the functional expression in (v,x) 
## to form a plot. So let's extract the left-hand side of the equation.  Recall how sympy thinks of the
## expression in a tree.
y = sp.Symbol('y', real=True)
print(sp.srepr(SOLt))
## We can see that the left-hand side is in the "Add" expression in the equality expression.  Let's see if we
## can access it. 
print("\nThe left-hand side of the equation: \n")
sp.pprint(SOLt.args[0])

SOLp = SOLt.args[0] ## this is what we will plot.
print("\nAnd replacing the functional expression f(x) by a variable y:\n")
SOLp = SOLp.subs(f(x), y)
sp.pprint(SOLp)
print("Hmm, sympy does not want to write the anti-derivative of 1/y as log(|y|)... we can fix this problem... ")

In [ ]:
## let's bring to a common denominator
SOLp = sp.together(SOLp)
sp.pprint(SOLp)

## and combine the logarithms
SOLp = sp.logcombine(SOLp, force=True)
sp.pprint(SOLp)

## you can find all the simplification routines described, here:
##  http://docs.sympy.org/dev/tutorial/simplification.html

print("Now we can plot!")

In [ ]:
from sympy.plotting import plot_implicit

curve1 = plot_implicit(sp.Eq(SOLp, 0), (x,-2,2), (y,-2,2), 
              adaptive=False, points=200, 
              xlabel="time-axis", ylabel="v-axis", show=False)

curve2 = plot_implicit(sp.Eq(SOLp, 1), (x,-2,2), (y,-2,2), 
              adaptive=False, points=200, 
              xlabel="time-axis", ylabel="v-axis", show=False)

curve1.extend(curve2)
curve1.show()

In [ ]:
## Or we could realize we could solve for x or y, rather than using the implicit plot. 
print("Solve for x: ")
print(sp.solve(sp.Eq(SOLp, sp.Symbol('c')), x))
print("Solve for y: ")
print(sp.solve(sp.Eq(SOLp, sp.Symbol('c')), y))

F = sp.solve(sp.Eq(SOLp, sp.Symbol('c')), y)[0]

In [ ]:
## this allows us to plot y as a function of x. 
sp.pprint(F)

F2 = F.subs(sp.Symbol('c'), 2)
print("After substitution: \n")
sp.pprint(F2)

In [ ]:
from matplotlib import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline 

import numpy as np
Lx = np.linspace(0, 5, 200)
F2n = sp.lambdify(x, F2, 'numpy')
plt.plot(Lx, F2n(Lx), 'r-')

## Solving ODES with sympy

At present, sympy's ode solving algorithm is basically a big database or *cookbook*-style database of formulas... that sympy can work with as it has algorithms to find anti-derivatives of functions.  

**Sympy has algorithms to solve:**

* First order ODEs that are: 
     - separable differential equations
     - differential equations whose coefficients homogeneous of the same order.
     - exact differential equations.
     - linear differential equations
     - Bernoulli differential equations.

* Second order ODEs that are:
    - Liouville differential equations.

* n-th order ODEs that are:
    - linear homogeneous differential equation with constant coefficients.
    - linear inhomogeneous differential equation with constant coefficients using the method of undetermined coefficients.
    - linear inhomogeneous differential equation with constant coefficients using the method of variation of parameters.

Sympy also has algorithms to solve some [PDEs](http://docs.sympy.org/latest/modules/solvers/pde.html), Delay Differential Equations [DDEs](http://users.ox.ac.uk/~clme1073/python/PyDDE/) and pretty much any other kind of differential equation you can imagine. 

A key issue to solving differential equations is *determining what kind of differential equation* one is trying to solve. Once you *know* a differential equation is in (or can be put in) **form $X$**, and if sympy has an algorithm to solve differential equations of **form $X$**, then sympy can quickly give the answer.  

**example: **

*Exact differential equations* are of the form:
$$ f(x,y) \frac{dy}{dx} + g(x,y) = 0$$
with 
$$ \frac{\partial f}{\partial x} = \frac{\partial g}{\partial y} $$

Sympy has a routine that uses heuristics, based on its algorithms to solve algebraic equations, that check to see if your differential equation is of a type that it knows how to solve.  These tests can be sophisticated or not, and depends on the ease-of-applicability of the method.  For example, things like linear or separable ODEs are relatively easy to recognise, but exact differential equations can sometimes be subtle, due to the flexibility of the *integrating factor* technique. 

$$y' = - \frac{3xy+y^2}{x^2+xy}$$

is an exact ODE... or at least, it can be made to be exact with an integrating factor. 



In [ ]:
y=sp.Function('y')

sp.classify_ode( sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), y(x) )

The *classify_ode* routine returns the list of "types" of ODEs that the differential equation satisfies, which sympy knows how to solve.  If you find the default solution that sympy gives you is not satisfactory (perhaps the algebra is too complicated) you can ask sympy to solve your differential equation using another method, using the *hint* argument.

In [ ]:
sp.pprint(sp.dsolve(sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), 
                    hint="1st_homogeneous_coeff_subs_indep_div_dep_Integral"))

In [ ]:
sp.pprint(sp.dsolve(sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), 
                    hint="1st_homogeneous_coeff_subs_dep_div_indep_Integral"))

In [ ]:
sp.pprint(sp.dsolve(sp.Eq( y(x).diff(x) + ((3*x*y(x)+y(x)**2)/(x**2+x*y(x))), 0 ), 
                    hint="1st_homogeneous_coeff_subs_indep_div_dep"))

Generally sympy puts the technique considered to give the *most pleasant to work with* solutions at the top of the *classify_ode* list. 

In [ ]:
## Let's go back to the skydiver problem
ODE = sp.Derivative(f(x), x) - k*f(x)**2 + g
sp.classify_ode( ODE, f(x))

In [ ]:
sp.pprint(sp.dsolve( ODE, hint="1st_power_series"))

In [ ]:
sp.pprint(sp.dsolve( ODE, hint="separable_Integral"))

In [ ]:
sp.pprint(sp.dsolve( ODE, hint="separable_Integral").doit() )